# This Jupyter notebook illustrates how to read data in from an external file 
## [notebook provides a simple illustration, users can easily use these examples to modify and customize  for their data storage scheme and/or preferred workflows] 


###Motion Blur Filtering: A Statistical Approach for Extracting  Confinement Forces & Diffusivity from a Single Blurred Trajectory

#####Author: Chris Calderon

Copyright 2015 Ursa Analytics, Inc.
Licensed under the Apache License, Version 2.0 (the "License");
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0



### Cell below loads the required modules and packages

In [1]:
%matplotlib inline 
#command above avoids using the "dreaded" pylab flag when launching ipython (always put magic command above as first arg to ipynb file)
import matplotlib.font_manager as font_manager
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as spo
import findBerglundVersionOfMA1 #this module builds off of Berglund's 2010 PRE parameterization (atypical MA1 formulation)
import MotionBlurFilter
import Ursa_IPyNBpltWrapper


##Now that required modules packages are loaded, set parameters for simulating "Blurred" OU trajectories.  Specific mixed continuous/discrete model:

\begin{align}
dr_t = & ({v}-{\kappa} r_t)dt + \sqrt{2 D}dB_t \\
\psi_{t_i} = & \frac{1}{t_E}\int_{t_{i}-t_E}^{t_i} r_s ds + \epsilon^{\mathrm{loc}}_{t_i}
\end{align}

###In above equations, parameter vector specifying model is: $\theta = (\kappa,D,\sigma_{\mathrm{loc}},v)$


###Statistically exact discretization of above for uniform time spacing  $\delta$  (non-uniform $\delta$ requires time dependent vectors and matrices below):

\begin{align}
r_{t_{i+1}} = & A + F r_{t_{i}} + \eta_{t_i} \\
\psi_{t_i} = & H_A + H_Fr_{t_{i-1}} + \epsilon^{\mathrm{loc}}_{t_i} + \epsilon^{\mathrm{mblur}}_{t_i} \\
\epsilon^{\mathrm{loc}}_{t_i} + & \epsilon^{\mathrm{mblur}}_{t_i} \sim  \mathcal{N}(0,R_i) \\
\eta_i \sim & \mathcal{N}(0,Q) \\
t_{i-1} = & t_{i}-t_E \\
 C = & cov(\epsilon^{\mathrm{mblur}}_{t_i},\eta_{t_{i-1}}) \ne 0
\end{align}


####Note:  Kalman Filter (KF) and Motion Blur Filter (MBF) codes estimate $\sqrt(2D)$ directly as "thermal noise" parameter

### For situations where users would like to read data in from external source, many options exist. 

####In cell below, we show how to read in a text file and process the data assuming the text file contains two columns:  One column with the 1D measurements and one with localization standard deviation vs. time estimates.  Code chunk below sets up some default variables (tunable values indicated by comments below).   Note that for multivariate signals, chunks below can readily be modified to process x/y or x/y/z measurements separately.  Future work will address estimating 2D/3D models with the MBF (computational [not theoretical] issues exists in this case);  however, the code currently provides diagnostic information to determine if unmodeled multivariate interaction effects are important (see main paper and Calderon, Weiss, Moerner, PRE 2014)

### Plot examles from other notebooks can be used to explore output within this notbook or another.   Next, a simple example of "Batch" processing is illustrated.

In [2]:
filenameBase='./ExampleData/MyTraj_' #assume all trajectory files have this prefix (adjust file location accordingly)

N=20 #set the number of trajectories to read.  
delta = 25./1000. #user must specify the time (in seconds) between observations.  code provided assumes uniform continuous illumination and 
#NOTE: in this simple example, all trajectories assumed to be collected with exposure time delta input above



#now loop over trajectories and store MLE results
resBatch=[] #variable for storing MLE output

#loop below just copies info from cell below (only difference is file to read is modified on each iteration of the loop)
for i in range(N):
    
    filei = filenameBase + str(i+1) + '.txt'
    print ''
    print '^'*100
    print 'Reading in file: ', filei
    #first load the sample data stored in text file.  here we assume two columns of numerica data (col 1 are measurements)
    data = np.loadtxt(filei)
    (T,ncol)=data.shape
    #above we just used a simple default text file reader;  however, any means of extracting the data and
    #casting it to a Tx2 array (or Tx1 if no localization accuracy info available) will work.



    ymeas = data[:,0]
    locStdGuess = data[:,1]  #if no localization info avaible, just set this to zero or a reasonable estimate of localization error [in nm]

    Dguess = 0.1 #input a guess of the local diffusion coefficient of the trajecotry to seed the MLE searches (need not be accurate)
    velguess = np.mean(np.diff(ymeas))/delta #input a guess of the velocity of the trajecotry to seed the MLE searches (need not be accurate)

    MA=findBerglundVersionOfMA1.CostFuncMA1Diff(ymeas,delta) #construct an instance of the Berglund estimator
    res = spo.minimize(MA.evalCostFuncVel, (np.sqrt(Dguess),np.median(locStdGuess),velguess), method='nelder-mead')

    #output Berglund estimation result.
    print 'Berglund MLE',res.x[0]*np.sqrt(2),res.x[1],res.x[-1]
    print '-'*100

    #obtain crude estimate of mean reversion parameter.  see Calderon, PRE (2013)
    kappa1 = np.log(np.sum(ymeas[1:]*ymeas[0:-1])/(np.sum(ymeas[0:-1]**2)-T*res.x[1]**2))/-delta

    #construct an instance of the MBF estimator
    BlurF = MotionBlurFilter.ModifiedKalmanFilter1DwithCrossCorr(ymeas,delta,StaticErrorEstSeq=locStdGuess)
    #use call below if no localization info avaible
    # BlurF = MotionBlurFilter.ModifiedKalmanFilter1DwithCrossCorr(ymeas,delta)

    parsIG=np.array([np.abs(kappa1),res.x[0]*np.sqrt(2),res.x[1],res.x[-1]]) #kick off MLE search with "warm start" based on simpler model
    #kick off nonlinear cost function optimization given data and initial guess
    resBlur = spo.minimize(BlurF.evalCostFunc,parsIG, method='nelder-mead')
    
    print 'parsIG for Motion Blur filter',parsIG
    print 'Motion Blur MLE result:',resBlur

    #finally evaluate diagnostic statistics at MLE just obtained
    loglike,xfilt,pit,Shist =BlurF.KFfilterOU1d(resBlur.x) 

    print np.mean(pit),np.std(pit)
    print 'crude assessment of model:  check above mean is near 0.5 and std is approximately',np.sqrt(1/12.)
    print 'statements above based on generalized residual U[0,1] shape'  
    print 'other hypothesis tests outlined which can use PIT sequence above outlined/referenced in paper.'
    
    #finally just store the MLE of the MBF in a list
    resBatch.append(resBlur.x)



^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Reading in file:  ./ExampleData/MyTraj_1.txt
Berglund MLE 0.424450138266 0.0410840106778 -0.000253509776551
----------------------------------------------------------------------------------------------------
parsIG for Motion Blur filter [  6.01896509e-01   4.24450138e-01   4.10840107e-02  -2.53509777e-04]
Motion Blur MLE result:   status: 0
    nfev: 196
 success: True
     fun: -1.1526156274680164
       x: array([  9.88732097e-01,   4.30329177e-01,   1.69786977e-02,
        -1.88788339e-04])
 message: 'Optimization terminated successfully.'
     nit: 110
0.512506169502 0.289227348189
crude assessment of model:  check above mean is near 0.5 and std is approximately 0.288675134595
statements above based on generalized residual U[0,1] shape
other hypothesis tests outlined which can use PIT sequence above outlined/referenced in paper.

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [3]:
#Summarize the results of the above N simulations 
#

resSUM=np.array(resBatch)
print 'Blur medians',np.median(resSUM[:,0]),np.median(resSUM[:,1]),np.median(resSUM[:,2]),np.median(resSUM[:,3])
print 'means',np.mean(resSUM[:,0]),np.mean(resSUM[:,1]),np.mean(resSUM[:,2]),np.mean(resSUM[:,3])
print 'std',np.std(resSUM[:,0]),np.std(resSUM[:,1]),np.std(resSUM[:,2]),np.std(resSUM[:,3])

print '^'*100 ,'\n\n'

Blur medians 1.27208496571 0.436395237852 0.0162757902936 0.0759079000468
means 1.32234434672 0.434561850166 0.0160360992158 0.0655553122287
std 0.380912845778 0.0234327797792 0.00299196795637 0.182354637945
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 


